<a href="https://colab.research.google.com/github/UrangaEsteban/progcon/blob/procesos_java/tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://repo1.maven.org/maven2/com/google/code/gson/gson/2.10.1/gson-2.10.1.jar --output "gson-2.10.1.jar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0   290k      0 --:--:-- --:--:-- --:--:--  290k


In [ ]:
!ls -lash

In [ ]:
!echo '{"A":["B","C","D"],"B":["E","F"],"D":["G"]}' > data.json

total 316K
4.0K drwxr-xr-x 1 root root 4.0K Apr  7 20:31 .
4.0K drwxr-xr-x 1 root root 4.0K Apr  7 18:58 ..
4.0K drwxr-xr-x 4 root root 4.0K Apr  6 13:38 .config
4.0K -rw-r--r-- 1 root root   44 Apr  7 20:17 data.json
4.0K -rw-r--r-- 1 root root 3.4K Apr  7 20:11 errors
280K -rw-r--r-- 1 root root 277K Apr  7 20:31 gson-2.10.1.jar
4.0K -rw-r--r-- 1 root root 3.6K Apr  7 20:33 ProcessStuff.java
   0 -rw-r--r-- 1 root root    0 Apr  7 20:11 salidaJava
4.0K drwxr-xr-x 1 root root 4.0K Apr  6 13:39 sample_data
4.0K -rw-r--r-- 1 root root  250 Apr  7 19:25 tp1_java.java
4.0K -rw-r--r-- 1 root root  250 Apr  7 19:25 tpt_java.java


In [ ]:
%%writefile ProcessStuff.java
import com.google.gson.Gson;
import com.google.gson.reflect.TypeToken;
import java.lang.reflect.Type;
import java.io.StringReader;
import java.io.PrintWriter;
import java.io.FileOutputStream;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.FileSystems;
import java.nio.file.FileSystem;
import java.util.*;

public class ProcessStuff {
    public static void main(String[] args) {
        if (args.length == 0) {
            System.err.println("You need to pass an argument");
            return;
        }
        String self = String.valueOf(args[0]);
        Gson gson = new Gson();
        Type customType = new TypeToken<HashMap<String, ArrayList<String>>>() {}.getType();
        HashMap<String, ArrayList<String>> dataMap = gson.fromJson(getFileData(DATA_FILE_NAME), customType); 
        
        info(self);

        ArrayList<Process> children = spawnChildren(dataMap, self);
        for(Process child : children) {
            try {
              child.waitFor();
            } catch (InterruptedException e) {
              System.err.println("Can spawn no more, we've been interrupted.");
            }
        }
        try {
          Thread.sleep(5000);
        } catch (InterruptedException e) {
          System.err.println("Can spawn no more, we've been interrupted.");
        }
    }

    public static String getFileData(String fileName) {
        String data = "";
        try {
            data = String.join("", Files.readAllLines(FileSystems.getDefault().getPath(fileName)));
        } catch (NullPointerException e) {
            e.printStackTrace();
        } catch (IOException e) {
            System.err.println("Couldn't read the file " + fileName);
            e.printStackTrace();
        }
        return data;
    }

    public synchronized static void info(String self) {
        ProcessHandle selfProcessHandle = ProcessHandle.current();
        try (
            PrintWriter out = new PrintWriter(new FileOutputStream(OUT, true), true);
        ) {
            out.print("I am: " + self + ". ");
            out.print("My PID is: " + selfProcessHandle.pid() + ". ");
            out.print("My PPID is: " + selfProcessHandle.parent().get().pid() + ".\n");
            out.flush();
        } catch (IOException e) {
            System.err.println("Couldn't open file: " + OUT);
            e.printStackTrace();
        } catch (NoSuchElementException e) {
            System.err.println("So sorry, I've no parent: " + OUT);
            e.printStackTrace();
        }
    }

    public static ArrayList<Process> spawnChildren(HashMap<String, ArrayList<String>> dataMap, String self) {
        ArrayList<Process> children = new ArrayList<>();
        if (dataMap.keySet().contains(self)) {
            for(String childValue : dataMap.get(self)) {
              try {
                children.add(new ProcessBuilder("java", "-cp", ".:gson-2.10.1.jar:gson-2.10.1.jar:", "ProcessStuff", childValue, "1>salidaJava", "2>errors").start());
              } catch (IOException e) {
                System.err.println("Couldn't spawn new process:  " + childValue);
              }
            }
        }
        return children;
    }

    final static String DATA_FILE_NAME = "data.json";
    final static String OUT = "miSalidaJava";
}

Overwriting ProcessStuff.java


In [ ]:
!rm miSalidaJava

In [ ]:
!javac -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff.java

In [ ]:
!nohup java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff "A" 1>salidaJava 2>errors &  

In [ ]:
%%writefile HelloWorld.java

public class HelloWorld {
    public static void main(String[] args) {
        System.out.println();
        try {
          Thread.sleep(25000);
        } catch (InterruptedException e) {
          System.err.println("Can spawn no more, we've been interrupted.");
        }
    }
}

Writing HelloWorld.java


In [ ]:
!nohup java HelloWorld.java 1>salidaJava 2>errors &  

In [ ]:
!ps -ef 

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 18:58 ?        00:00:01 /sbin/docker-init -- /datala
root           7       1  0 18:58 ?        00:00:24 /tools/node/bin/node /datala
root          18       0  0 18:58 ?        00:00:01 tail -n +0 -F /root/.config/
root          29       7  0 18:58 ?        00:00:06 /bin/bash -e /usr/local/cola
root          31       1  0 18:58 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          33      31  0 18:58 ?        00:00:04 /usr/colab/bin/kernel_manage
root          59       7  0 18:58 ?        00:00:08 [python3] <defunct>
root          60       7  0 18:58 ?        00:00:00 python3 /usr/local/bin/colab
root          80       7  0 18:58 ?        00:00:24 /usr/bin/python3 /usr/local/
root          82       7  0 18:58 ?        00:00:11 /usr/local/bin/dap_multiplex
root        6297      80  0 19:24 ?        00:01:04 /usr/bin/python3 -m ipykerne
root        6342       1  0 19:24 ?        00:00:24 /usr/bin/p

In [ ]:
!pstree -pc 53922                                          

java(53922)─┬─{java}(53923)
            ├─{java}(53924)
            ├─{java}(53925)
            ├─{java}(53926)
            ├─{java}(53927)
            ├─{java}(53928)
            ├─{java}(53929)
            ├─{java}(53930)
            ├─{java}(53931)
            ├─{java}(53932)
            ├─{java}(53933)
            ├─{java}(53934)
            ├─{java}(53935)
            ├─{java}(53936)
            ├─{java}(53937)
            ├─{java}(53938)
            ├─{java}(53943)
            └─{java}(53944)


In [ ]:
!cat errors

In [ ]:
!cat miSalidaJava

I am: A. My PID is: 54621. My PPID is: 1.
I am: C. My PID is: 54640. My PPID is: 54621.
I am: D. My PID is: 54643. My PPID is: 54621.
I am: B. My PID is: 54638. My PPID is: 54621.
I am: E. My PID is: 54712. My PPID is: 54638.
I am: G. My PID is: 54696. My PPID is: 54643.
I am: F. My PID is: 54721. My PPID is: 54638.
